In [15]:
pip install --trusted-host=artifactory.oci.oraclecorp.com -i https://artifactory.oci.oraclecorp.com/api/pypi/global-dev-pypi/simple -U oci==2.158.0+394

Looking in indexes: https://artifactory.oci.oraclecorp.com/api/pypi/global-dev-pypi/simple
     ---------------------------------------- 0.0/32.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/32.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/32.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/32.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/32.1 MB ? eta -:--:--
     ---------------------------------------- 0.3/32.1 MB ? eta -:--:--
      --------------------------------------- 0.5/32.1 MB 1.0 MB/s eta 0:00:31
      --------------------------------------- 0.5/32.1 MB 1.0 MB/s eta 0:00:31
     - -------------------------------------- 1.0/32.1 MB 1.5 MB/s eta 0:00:21
     - -------------------------------------- 1.6/32.1 MB 1.6 MB/s eta 0:00:19
     -- ------------------------------------- 2.1/32.1 MB 1.7 MB/s eta 0:00:18
     -- ------------------------------------- 2.1/32.1 MB 1.7 MB/s eta 0:00:18
   

  DEPRECATION: Wheel filename 'oci-1.3.12+preview.1+47-py2.py3-none-any.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): 'oci-1.3.12+preview.1+47-py2.py3-none-any'
  
   pip 25.3 will enforce this behaviour change. A possible replacement is to rename the wheel to use a correctly normalised name (this may require updating the version in the project metadata). Discussion can be found at https://github.com/pypa/pip/issues/12938
  DEPRECATION: Wheel filename 'oci-1.3.14+preview.1+51-py2.py3-none-any.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): 'oci-1.3.14+preview.1+51-py2.py3-none-any'
  
   pip 25.3 will enforce this behaviour change. A possible replacement is to rename the wheel to use a correctly normalised name (this may require updating the version in the project metadata). Discussion can be found at https://github.

In [23]:
from vision_service_python_client import ai_service_vision_client
from vision_service_python_client.models import CreateStreamJobDetails, CreateStreamSourceDetails, \
    RtspSourceDetails, ObjectStorageOutputLocation, PublicStreamNetworkAccessDetails
import oci, time

In [24]:
CONFIG_PROFILE = "DEFAULT"
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaarsssgsb3dqeibesnmbaaqkvomkpl5sua26wfuz5povnvohdw2ekq"
STREAM_JOB_DISPLAY_NAME = "Python SDK Test"
CAMERA_URL="rtsp://64.181.159.98:443/stream"
NAMESPACE= "frpj5kvxryk1"
BUCKET= "stream_vision_bucket"
PREFIX= "testing_person_min"
FEATURES = [
        {
            "featureType": "OBJECT_DETECTION",
            
            "minConfidence": 0.35
            # "trackingTypes":[
            #     {
            #         "objects": ["face"],
            #         "maxResults": 5,
            #         "shouldReturnLandmarks": True,
            #     }
            # ]
        }
    ]
print("done")

done


In [25]:
config = oci.config.from_file('~/.oci/config', profile_name=CONFIG_PROFILE)
endpoint = ""
token_file = config['security_token_file']
with open(token_file, 'r') as f:
   token = f.read()
private_key = oci.signer.load_private_key_from_file(config['key_file'])
signer = oci.auth.signers.SecurityTokenSigner(token, private_key)
ai_service_vision_client = ai_service_vision_client.AIServiceVisionClient(config=config, signer=signer, service_endpoint=endpoint)
print("done")

done


In [26]:
create_stream_source_details = CreateStreamSourceDetails()

stream_source_details = RtspSourceDetails()
stream_source_details.camera_url = CAMERA_URL
stream_source_details.stream_network_access_details = PublicStreamNetworkAccessDetails()
create_stream_source_details.stream_source_details = stream_source_details
create_stream_source_details.compartment_id = COMPARTMENT_ID

# Send request for adding the device and parse the response
print("done")

done


In [27]:
stream_source = ai_service_vision_client.create_stream_source(create_stream_source_details)
source_id = stream_source.data.id
print(source_id)
while stream_source.data.lifecycle_state != 'ACTIVE':
    stream_source = ai_service_vision_client.get_stream_source(source_id)
    time.sleep(2)
print("done")

ocid1.aivisionstreamsource.oc1..aaaaaaaaechtt4bpecmiiiony4gvmnyu7a3a43oziwclqxorm4z4kzp6zm2q
done


In [28]:
stream_source = ai_service_vision_client.get_stream_source(source_id)
print(stream_source.data)

{
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaarsssgsb3dqeibesnmbaaqkvomkpl5sua26wfuz5povnvohdw2ekq",
  "defined_tags": {},
  "display_name": "aivisionstreamsource20251113075113",
  "freeform_tags": {},
  "id": "ocid1.aivisionstreamsource.oc1..aaaaaaaaechtt4bpecmiiiony4gvmnyu7a3a43oziwclqxorm4z4kzp6zm2q",
  "lifecycle_state": "ACTIVE",
  "stream_source_details": {
    "camera_url": "rtsp://64.181.159.98:443/stream",
    "secret_id": null,
    "source_type": "RTSP",
    "stream_network_access_details": {
      "stream_access_type": "PUBLIC"
    }
  },
  "system_tags": {},
  "time_created": "2025-11-13T07:51:13.253000+00:00",
  "time_updated": "2025-11-13T07:51:13.253000+00:00"
}


In [29]:
create_stream_job_details = CreateStreamJobDetails()
create_stream_job_details.stream_source_id = source_id

# For Object Tracker (Only "face" supported as of now) - commend if not required
create_stream_job_details.features = FEATURES
# print(create_stream_job_detai)

output_location = ObjectStorageOutputLocation()
output_location.namespace_name = NAMESPACE
output_location.bucket_name = BUCKET
output_location.prefix = PREFIX
# Choose output location in object storage, make sure you have created the bucket
create_stream_job_details.stream_output_location = output_location

create_stream_job_details.compartment_id = COMPARTMENT_ID
create_stream_job_details.display_name = STREAM_JOB_DISPLAY_NAME


In [30]:
stream_job = ai_service_vision_client.create_stream_job(create_stream_job_details=create_stream_job_details)
job_id = stream_job.data.id
print(job_id)
while stream_job.data.lifecycle_state == 'CREATING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(10)


ocid1.aivisionstreamjob.oc1..aaaaaaaawcophuezqz7kewd2g4dioaxh4cbc5laf474enhfc4f5ue3yaycqa


In [33]:
print(ai_service_vision_client.get_stream_job(job_id).data)

{
  "agent_participant_id": null,
  "compartment_id": "ocid1.compartment.oc1..aaaaaaaarsssgsb3dqeibesnmbaaqkvomkpl5sua26wfuz5povnvohdw2ekq",
  "defined_tags": {},
  "display_name": "Python SDK Test",
  "features": [
    {
      "feature_type": "UNKNOWN_ENUM_VALUE"
    }
  ],
  "freeform_tags": {},
  "id": "ocid1.aivisionstreamjob.oc1..aaaaaaaawcophuezqz7kewd2g4dioaxh4cbc5laf474enhfc4f5ue3yaycqa",
  "lifecycle_details": null,
  "lifecycle_state": "UPDATING",
  "stream_output_location": {
    "bucket_name": "stream_vision_bucket",
    "namespace_name": "frpj5kvxryk1",
    "obo_token": null,
    "output_location_type": "OBJECT_STORAGE",
    "prefix": "testing_person_min"
  },
  "stream_source_id": "ocid1.aivisionstreamsource.oc1..aaaaaaaaechtt4bpecmiiiony4gvmnyu7a3a43oziwclqxorm4z4kzp6zm2q",
  "system_tags": {},
  "time_created": "2025-11-13T07:51:23.140000+00:00",
  "time_updated": "2025-11-13T07:51:25.811000+00:00"
}


In [32]:
print("Calling startStream job at ", time.time())
ai_service_vision_client.start_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'UPDATING':
    print("updating")
    print(stream_job.data.lifecycle_state)
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)
print("Complete")


Calling startStream job at  1763020285.530762
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING
updating
UPDATING


KeyboardInterrupt: 

In [34]:
ai_service_vision_client.stop_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'UPDATING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)

ServiceError: {'target_service': 'ai_service_vision', 'status': 405, 'code': 'MethodNotAllowed', 'opc-request-id': '76AFD9D5F3104BB396A587347A013AA2/15A93B1B18D9FAFB9993156949414455/5563022A3161180CFF0662C43F4F36D0', 'message': 'Cannot stop already Updating resource', 'operation_name': 'stop_stream_job', 'timestamp': '2025-11-13T08:15:21.312952+00:00', 'client_version': 'Oracle-PythonSDK/2.156.1+preview.1.257', 'request_endpoint': 'POST https://vision.aiservice.us-ashburn-1.oci.oraclecloud.com/20220125/streamJobs/ocid1.aivisionstreamjob.oc1..aaaaaaaawcophuezqz7kewd2g4dioaxh4cbc5laf474enhfc4f5ue3yaycqa/actions/stop', 'logging_tips': 'To get more info on the failing request, refer to https://docs.oracle.com/en-us/iaas/tools/python/latest/logging.html for ways to log the request/response details.', 'troubleshooting_tips': 'See https://docs.oracle.com/iaas/Content/API/References/apierrors.htm#apierrors_405__405_methodnotallowed for more information about resolving this error. If you are un

In [ ]:
print(job_id)
ai_service_vision_client.delete_stream_job(job_id)
stream_job = ai_service_vision_client.get_stream_job(job_id)
while stream_job.data.lifecycle_state == 'DELETING':
    stream_job = ai_service_vision_client.get_stream_job(job_id)
    time.sleep(2)

In [ ]:
print(stream_source.data.id)
ai_service_vision_client.delete_stream_source(stream_source.data.id)
stream_job = ai_service_vision_client.get_stream_job(job_id)

while stream_job.data.lifecycle_state == 'DELETING':
    stream_job = ai_service_vision_client.get_stream_source(stream_source.data.id)
    time.sleep(2)